In [38]:
import re
import os
import subprocess 
import pandas as pd 

def izdvoji_nisku_iz_zaglavlja(zaglavlje):

    zaglavlje = zaglavlje.lstrip('>')
    match = re.match(r"^[^[]+", zaglavlje) 
    if match:
        return match.group(0).strip()  
    return zaglavlje.strip() 


In [39]:
import os

def podeli_fasta_vsl2b(fasta_fajl, izlazni_direktorijum):
    """
    Podeli FASTA fajl na pojedinačne fajlove, uklanjajući zaglavlja za VSL2B,
    ali čuvajući zaglavlja u zasebnom CSV fajlu.

    Args:
        fasta_fajl: Putanja do ulaznog FASTA fajla.
        izlazni_direktorijum: Putanja do izlaznog direktorija.
    """

    if not os.path.exists(izlazni_direktorijum):
        os.makedirs(izlazni_direktorijum)

    # Kreiraj CSV fajl za pohranu zaglavlja
    csv_fajl = os.path.join(izlazni_direktorijum, "zaglavlja.csv")
    with open(csv_fajl, 'w') as csv:
        csv.write("Naziv_proteina\n")

    with open(fasta_fajl, 'r') as fajl:
        sekvenca = ""
        broj_fajlova = 0

        for linija in fajl:
            linija = linija.strip()
            if linija.startswith(">"):
                if sekvenca.strip():  # Ako postoji prethodna sekvenca, spremi je u fajl
                    naziv_proteina = izdvoji_nisku_iz_zaglavlja(prethodno_zaglavlje)
                    izlazni_fajl = os.path.join(izlazni_direktorijum, f"{naziv_proteina}.fasta")
                    with open(izlazni_fajl, 'w') as izlaz:
                        izlaz.write(sekvenca)  # Upisuje samo sekvencu, bez zaglavlja

                    # Upisi naziv proteina u CSV fajl
                    with open(csv_fajl, 'a') as csv:
                        csv.write(f"{naziv_proteina}\n")

                sekvenca = ""  # Resetuj sekvencu za novu
                prethodno_zaglavlje = linija  # Spremi trenutno zaglavlje
            else:
                sekvenca += linija

        # Spremi posljednju sekvencu
        if sekvenca.strip():
            naziv_proteina = izdvoji_nisku_iz_zaglavlja(prethodno_zaglavlje)
            izlazni_fajl = os.path.join(izlazni_direktorijum, f"{naziv_proteina}.fasta")
            with open(izlazni_fajl, 'w') as izlaz:
                izlaz.write(sekvenca)

            # Upisi naziv proteina u CSV fajl
            with open(csv_fajl, 'a') as csv:
                csv.write(f"{naziv_proteina}\n")

    print(f"FASTA fajl podeljen u {broj_fajlova} manjih fajlova u {izlazni_direktorijum}.")

In [40]:
def pokreni_vsl2b_na_fajlovima(izlazni_direktorijum):
    """
    Pokreće VSL2B za svaki pojedinačni FASTA fajl u datom direktorijumu. U prosleđivanju fajlova, preskoči zaglavlja.
    """
    rezultati = []
    fasta_fajlovi = sorted([f for f in os.listdir(izlazni_direktorijum) if f.endswith('.fasta')])
    
    for i, fasta_fajl in enumerate(fasta_fajlovi, 1):
        putanja_fajla = os.path.join(izlazni_direktorijum, fasta_fajl)
        print(f"Obradjuje se {i}/{len(fasta_fajlovi)}: {fasta_fajl}")
                
        try:
            rezultat = subprocess.run(
                ['java', '-jar', '/home/pc/Desktop/VSL2/VSL2.jar', f'-s:{putanja_fajla}'], 
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                text=True
            )
            if rezultat.stderr:
                print(f"Upozorenje: {rezultat.stderr.strip()}")
            rezultati.append((fasta_fajl, rezultat.stdout))
        except Exception as e:
            print(f"Greška pri obradi {fasta_fajl}: {e}")
    
    return rezultati


In [41]:
# Parsiranje izlaza iz VSL2B
def parse_vsl2b_output(vsl2b_output):
    """
    Parsira izlaz iz VSL2B programa i klasifikuje svaku poziciju u sekvenci kao 'uredjenu' ili 'neuredjenu'.
    """
    results = []
    for line in vsl2b_output.splitlines():
        line = line.strip()
        # Preskoči linije koje nisu podaci o sekvenci
        if not line or line.startswith("Prediction Scores") or line.startswith("VSL2") or line.startswith("Center"):
            continue
        parts = line.split()
        if len(parts) >= 3:
            try:
                pos = int(parts[0])  # Pozicija
                amino_acid = parts[1]  # Aminokiselina
                score = float(parts[2])  # VSL2B score
                classification = "uređena" if score <= 0.5 else "neuređena"
                results.append((pos, amino_acid, score, classification))
            except ValueError:
                # Ignoriši linije koje ne mogu da se parsiraju
                print(f"Skipping invalid line: {line}")
                continue
    return results

In [42]:
def kreiraj_dataframe_iz_rezultata(rezultati):
    """
    Kreira DataFrame sa informacijama o sekvencama, uključujući pozicije,
    aminokiseline, skorove, klasifikacije i naziv proteina.
    Sekvence se grupišu prema tipu proteina ('nucleoprotein', 'spike', 'matrix').
    """
    redovi = []
    for ime_fajla, izlaz in rezultati:
        # Izdvajanje naziva proteina iz naziva fajla (zamišljeno da naziv proteina bude u imenu fajla)
        naziv_proteina = ime_fajla.replace(".fasta", "")  # Koristi naziv fajla kao naziv proteina
        parsirani_rezultati = parse_vsl2b_output(izlaz)
        for pozicija, aminokiselina, skor, klasifikacija in parsirani_rezultati:
            redovi.append({
                "Naziv proteina": naziv_proteina.replace(".fasta", ""),
                "Pozicija": pozicija,
                "Aminokiselina": aminokiselina,
                "VSL2b rezultat": skor,
                "Klasifikacija": klasifikacija,
            })
    
    # Kreiranje DataFrame-a
    df = pd.DataFrame(redovi)
    
    # Funkcija za određivanje prioriteta grupisanja
    def prioritet_grupisanja(naziv):
        naziv = naziv.lower()
        if "nucleoprotein" in naziv or "nucleocapsid" in naziv:
            return 1
        elif "spike" in naziv:
            return 2
        elif "matrix" in naziv:
            return 3

    # Grupisanje i očuvanje redosleda unutar grupa
    df["Grupa"] = df["Naziv proteina"].apply(prioritet_grupisanja)
    df["Originalni redosled"] = range(len(df))  # Zadržavanje izvornog redosleda
    df.sort_values(by=["Grupa", "Originalni redosled"], inplace=True)
    df.drop(columns=["Grupa", "Originalni redosled"], inplace=True)  # Uklanjanje pomoćnih kolona

    return df


In [43]:
def glavna_funkcija_sa_podelom():
    """
    Glavna funkcija za podelu FASTA fajla, pokretanje IUPred3 i generisanje tabele.
    """
    fasta_fajl = "/home/pc/Desktop/Istrazivanje-podataka-2/Podaci/izdvojeno_proteini_ebola.fasta"
    izlazni_dir = "/home/pc/Desktop/Istrazivanje-podataka-2/VSL2_parsiranje/sekvence_ebola_2"
    
    # Podela velikog FASTA fajla
    podeli_fasta_vsl2b(fasta_fajl, izlazni_dir)
    
    # Pokretanje IUPred3 na podeljenim fajlovima
    rezultati = pokreni_vsl2b_na_fajlovima(izlazni_dir)
    
    # Kreiranje DataFrame-a
    df = kreiraj_dataframe_iz_rezultata(rezultati)
    return df


In [44]:
# Generisanje DataFrame-a
df_rezultati = glavna_funkcija_sa_podelom()

# Prikazivanje prvih 5 redova
print(df_rezultati.head())

# Čuvanje rezultata u CSV fajl
df_rezultati.to_csv("/home/pc/Desktop/Istrazivanje-podataka-2/VSL2_parsiranje/vsl2b_rezultati_ebola.csv", index=False)


FASTA fajl podeljen u 0 manjih fajlova u /home/pc/Desktop/Istrazivanje-podataka-2/VSL2_parsiranje/sekvence_ebola_2.
Obradjuje se 1/2100: AAD14583.1 |matrix protein.fasta
Obradjuje se 2/2100: AAD14585.1 |virion spike glycoprotein precursor.fasta
Obradjuje se 3/2100: AAD14590.1 |nucleoprotein.fasta
Obradjuje se 4/2100: AAG40164.1 |nucleoprotein NP.fasta
Obradjuje se 5/2100: AAM76031.1 |nucleoprotein NP.fasta
Obradjuje se 6/2100: AAM76033.1 |membrane-associated matrix protein VP40.fasta
Obradjuje se 7/2100: AAN04448.1 |nucleoprotein.fasta
Obradjuje se 8/2100: AAN04450.1 |matrix protein VP40.fasta
Obradjuje se 9/2100: AAN37504.1 |nucleoprotein NP.fasta
Obradjuje se 10/2100: AAN37505.1 |second nucleoprotein VP35.fasta
Obradjuje se 11/2100: AAN37509.1 |nucleoprotein VP30.fasta
Obradjuje se 12/2100: AAQ55045.1 |nucleoprotein.fasta
Obradjuje se 13/2100: AAQ55047.1 |matrix protein.fasta
Obradjuje se 14/2100: AAQ55048.1 |virion spike glycoprotein precursor.fasta
Obradjuje se 15/2100: AAU43883.1 